In [1]:
import sys
import os

sys.path.append("../Pre_processing")

from Data_Preparation_Library import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [11]:
!ls /scratch/leuven/319/vsc31950/output_files/

128_128  32_32


In [3]:
#selectedBatches=["6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
maxBatchId = 1
selectedBatches=[str(i) for i in range(maxBatchId)]

finput = str(image_size)+"_"+str(image_size)

batch_data_object = []
for i in selectedBatches:
    with open(os.path.join(temp_path,finput,'full_data_object_' + i + '.p'), 'rb') as handle:
        batch_data_object+=pickle.load(handle)

data_train = batch_data_object[0:int(len(batch_data_object)*2/3)]
data_test  = batch_data_object[int(len(batch_data_object)*2/3):]

In [4]:
def flux(d):
  # TODO: get flux_i from model
    if d.i_image != None:
        flux_i = sum(sum(d.i_image))
    else:
        flux_i = 0
    flux_g = sum(sum(d.g_image))
    #return flux_i+flux_g, flux_g-flux_i
    return flux_g

In [14]:
fresult = "results_"+str(image_size)+"_on_"+str(image_size)
postfix = "_all"
with open(os.path.join(output_path,fresult,"encoder_results_train_v2_" + postfix),'rb') as handle:
    encoded_imgs1=pickle.load(handle)
    encoded_imgs_reshaped_train = [e.reshape(-1,1) for e in encoded_imgs1]

with open(os.path.join(output_path,fresult,"encoder_results_test_v2_"  + postfix),'rb') as handle:
    encoded_imgs2=pickle.load(handle)
    encoded_imgs_reshaped_test = [e.reshape(-1,1) for e in encoded_imgs2]


In [15]:
#def pca_feature(d):
    
#  return [v1_i,v2_i,v1_i/v2_i,v2_i/v1_i, v1_g,v2_g,v1_g/v2_g,v2_g/v1_g]


In [16]:
from skimage.measure import label
from scipy.ndimage.morphology import distance_transform_edt
from sklearn.decomposition import PCA
def clean_stars(img):
    mask = img >= np.percentile(img, 96)
    lbl = label(mask, background=0)
    center_label = lbl[image_size//2,image_size // 2]
    galaxy = lbl == center_label
    stars = (lbl != center_label) & (lbl != 0) 
    image_clean = img / (1+ 5* stars + distance_transform_edt(~galaxy))
    return mask, galaxy, lbl, image_clean
def get_features(img):
    f = {}
    __, galaxy, lbl, image_clean = clean_stars(img)
    f['flux'] = img[galaxy].sum()
    gxy = np.array(np.nonzero(galaxy)).T
    gxy_centered = gxy - gxy.mean(axis=0, keepdims=True)
    pca = PCA().fit(gxy_centered)
    f['pca_eigen1'] = pca.explained_variance_[0]
    f['pca_eigen2'] = pca.explained_variance_[1]
    f['ellipticity'] = f['pca_eigen1'] / f['pca_eigen2']
    f['minx'] = gxy[:,0].min()
    f['maxx'] = gxy[:,0].max()
    f['miny'] = gxy[:,1].min()
    f['maxy'] = gxy[:,1].max()
    distances = np.linalg.norm(gxy_centered, axis=1)
    f['max_dist'] = distances.max()
    f['median_dist'] = np.median(distances)
    f['2d_first_moment'] = (distances * distances * img[np.nonzero(galaxy)]).sum()
    f['max_intensity'] = img[galaxy].max()
    return f

In [20]:
# accessible in the class: size of image, maximum of image, normalization factor of g + fitted value of normalization factor for i
# define a function to get the training and testing dataset
no_star_features = ['flux', 'pca_eigen1', 'pca_eigen2', 'ellipticity', 'max_dist',
                    'median_dist', '2d_first_moment', 'max_intensity']
def obtain_train_data():
    train_features = []
    train_labels = []
    for i in range(len(data_train)):
        if data_train[i].logMstar!=-99:
            e = encoded_imgs_reshaped_train[i]
            
            d = data_train[i]
            #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            no_star_f = get_features(d.g_image_resized)
            img_max=d.g_image.max()
            train_features.append(np.append(e,np.array([f,size_1,size_2,img_max] + [no_star_f[feat_name] for feat_name in no_star_features])))
            train_labels.append(d.logMstar)
    return train_features, train_labels

train_features = np.asarray(obtain_train_data()[0])
train_labels = np.asarray(obtain_train_data()[1])

print(train_features.shape, train_labels.shape)
print(train_features[0])

(659, 268) (659,)
[  1.90987945e-01   8.84844005e-01   0.00000000e+00   6.79106936e-02
   1.94844604e-01   8.77789378e-01   0.00000000e+00   1.86601095e-02
   1.84904635e-01   8.52527380e-01   0.00000000e+00   0.00000000e+00
   3.14054757e-01   7.65758157e-01   0.00000000e+00   0.00000000e+00
   3.94813389e-01   7.81931818e-01   0.00000000e+00   0.00000000e+00
   3.61049801e-01   8.55804801e-01   0.00000000e+00   0.00000000e+00
   2.59723842e-01   8.78644705e-01   0.00000000e+00   2.81448197e-02
   2.21107483e-01   8.80904675e-01   0.00000000e+00   5.44141307e-02
   1.86676353e-01   8.84442687e-01   0.00000000e+00   6.30877316e-02
   1.85942948e-01   8.75049472e-01   0.00000000e+00   0.00000000e+00
   1.71866149e-01   8.39093864e-01   0.00000000e+00   0.00000000e+00
   2.96946794e-01   7.31972516e-01   0.00000000e+00   0.00000000e+00
   3.75833005e-01   7.51925766e-01   0.00000000e+00   0.00000000e+00
   3.89844328e-01   8.42040360e-01   0.00000000e+00   0.00000000e+00
   2.52513677e-0

In [92]:
def obtain_test_data():
    test_features = []
    test_labels = []
    for j in range(len(data_test)):
        if data_test[j].logMstar!=-99:
            e = encoded_imgs_reshaped_test[j]
            d = data_test[j]

        #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            test_features.append(np.append(e,np.array([f,size_1,size_2,img_max])))
            test_labels.append(d.logMstar)

    return test_features, test_labels

test_features = np.asarray(obtain_test_data()[0])
test_labels = np.asarray(obtain_test_data()[1])

print(test_features.shape, test_labels.shape)

(325, 4100) (325,)


In [93]:
# ## transform features
# train_features = StandardScaler().fit_transform(train_features)
# test_features = StandardScaler().transform(test_features)
# #features = MinMaxScaler().fit_transform(features)

X_scaler = StandardScaler()
train_features = X_scaler.fit_transform(train_features)
test_features = X_scaler.transform(test_features)

print(train_features[:5], test_features[:5])

[[-0.46823421 -0.5080603   0.02156935 ..., -0.08512842 -0.08512842
  -0.31407447]
 [-0.37778186  0.01002213  0.36101057 ...,  1.97956789  1.97956789
  -0.3734218 ]
 [-0.31466022 -0.04653447  0.02059651 ...,  1.35231838  1.35231838
   1.8731427 ]
 [-0.34544549  0.51801735  0.59461292 ..., -0.45102397 -0.45102397
  -0.4427597 ]
 [ 0.5452832   1.31677986  0.74994534 ..., -0.56863325 -0.56863325
  -0.42390013]] [[ 0.15133955 -1.55998065 -0.05018915 ...,  0.01941317  0.01941317
  -0.41906929]
 [-0.31818065  0.05129635  0.23653529 ...,  1.888094    1.888094
  -0.11246488]
 [ 0.8351674   1.77883233  1.02854044 ..., -0.49022706 -0.49022706
  -0.44070063]
 [-0.48722797 -0.36853051  0.03476775 ...,  0.01941317  0.01941317
  -0.30787221]
 [-0.51023902 -1.52314199 -1.47030972 ..., -0.52943016 -0.52943016
  -0.42655866]]


In [94]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [150]:
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_dim=train_features.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(optimizer='Adadelta',
              loss='mse',
              metrics=['mse'])

In [151]:
model.fit(train_features, train_labels, epochs=100, batch_size=32,shuffle=True,
                validation_data=(test_features, test_labels))

Train on 659 samples, validate on 325 samples
Epoch 1/100
659/659 [==============================] - 0s - loss: 116.7597 - mean_squared_error: 116.7597 - val_loss: 111.0752 - val_mean_squared_error: 111.0752
Epoch 2/100
659/659 [==============================] - 0s - loss: 102.1920 - mean_squared_error: 102.1920 - val_loss: 98.6557 - val_mean_squared_error: 98.6557
Epoch 3/100
659/659 [==============================] - 0s - loss: 90.6146 - mean_squared_error: 90.6146 - val_loss: 87.7422 - val_mean_squared_error: 87.7422
Epoch 4/100
659/659 [==============================] - 0s - loss: 80.9058 - mean_squared_error: 80.9058 - val_loss: 78.5043 - val_mean_squared_error: 78.5043
Epoch 5/100
659/659 [==============================] - 0s - loss: 72.3109 - mean_squared_error: 72.3109 - val_loss: 70.6543 - val_mean_squared_error: 70.6543
Epoch 6/100
659/659 [==============================] - 0s - loss: 64.9769 - mean_squared_error: 64.9769 - val_loss: 64.0039 - val_mean_squared_error: 64.0039


In [152]:
type(test_predict),type(test_labels[1])

(list, numpy.float64)

In [153]:
from sklearn.metrics import mean_squared_error

test_predict = []
test_labels_2=[]
for i in range(100):
    if data_test[i].logMstar!=-99:
        test_predict.append(model.predict(test_features[i].reshape(1,-1))[0][0])
        test_labels_2.append(test_labels[i])
print(len(test_predict))
print(np.sqrt(mean_squared_error(np.asarray(test_predict), np.asarray(test_labels_2))))


100
0.471828023804
